<a href="https://colab.research.google.com/github/hyxxnii/Tave-6th-Project/blob/master/NIPA%20%EB%B3%B8%EC%84%A0%20-%20%EC%9C%A1%EA%B5%B0%20%EB%AF%BC%EC%9B%90%20%EB%B6%84%EB%A5%98%20%EB%AA%A8%EB%8D%B8_try1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 육군 SNS 계정 게시글 댓글 카테고리 분류(긍정 / 부정 / 중립)
- 채점: f1-score(macro)
-데이터 구조
    - comment: 댓글내용
    - label: comment 중립(0)/긍정(1)/부정(2)
- 제출 양식: 컬럼 정보 없이 예측된 tag값만 존재하는 tsv파일

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

train_path = '../data/.train/.task145ㅁ/data/train.tsv'
test_path = '../data/.train/.task145/data/test.tsv'

In [ ]:
train_df = pd.read_csv(train_path, sep='\t')
train_df

In [ ]:
def read_documents(filename):
    with open(filename, encoding="utf-8") as f: # 윈도우는 꼭 encoding 해줘야 함
        documents = [line.split('\t') for line in f.read().splitlines()] 
        documents = documents[1:] # 첫번째 줄이 카테고리 이름적혀있는 줄이라서 날려버림
        
    return documents

test = read_documents(test_path)
test_df = pd.DataFrame(test)
test_df.columns = ["comment", "tag"]
test_df

In [ ]:
test_df['tag'] = test_df['tag'].astype('int')

In [ ]:
import konlpy 
from konlpy.tag import Mecab, Kkma, Okt, Komoran
import json
import os
import re
from pprint import pprint

def text_cleaning(doc):
    # 한국어를 제외한 글자를 제거하는 함수.
    doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
    return doc

def define_stopwords(path):
    SW = set()
    with open(path, encoding='utf-8') as f:
        for word in f:
            SW.add(word)
            
    return SW

def text_tokenizing(doc):
    return [word for word in mecab.nouns(doc) if word not in SW and len(word) > 1]

#### 불러온 데이터를 품사 태그를 붙여서 토크나이징

In [ ]:
import konlpy 
from konlpy.tag import Mecab, Kkma, Okt, Komoran
import json
import os
import re
from pprint import pprint

# 형태소 분석기 불러오기
okt = Okt()
mecab = Mecab()

SW = define_stopwords("./stopwords-ko.txt")

#  os.path.exists(): 어떤 파일이 디렉토리에 있는지 확인하는 library
if os.path.exists("train_docs.json"):
    # train_docs.json:형태소 분석기 돌려서 나온 결과를 json형태로 저장한 파일
    # 있으면 불러오기
    with open("train_docs.json", encoding='utf-8') as f:
        train_data = json.load(f)
    
else:
    # 없으면 형태소 분석해야지
    # 리뷰 리스트, 레이블 가져오기
    train_data = [(text_tokenizing(line[0]), line[1]) for line in train_df.values if text_tokenizing(line[0])]
    
    with open("train_docs.json", 'w', encoding='utf-8') as f:
        json.dump(train_data, f, ensure_ascii=False, indent='\t')
        # ensure_ascii=False: 아스키코드 보존할거냐
        # indent='\t': 저장도 탭 기준으로
        
if os.path.exists("test_docs.json"):
    with open("test_docs.json", encoding='utf-8') as f:
        test_data = json.load(f)
        
else:
    test_data = [(text_tokenizing(line[0]), line[1]) for line in test_df.values if text_tokenizing(line[0])]
    
    with open("test_docs.json", "w", encoding='utf-8') as f:
        json.dump(test_data, f, ensure_ascii=False, indent='\t')
        
pprint(train_data[0])
pprint(test_data[0])

In [ ]:
print(len(train_df) - len(train_data))
print(len(test_df) - len(test_data))
# 정제하고 사라진 데이터 갯수

In [ ]:
import nltk

total_tokens = [token for doc in train_data for token in doc[0]]
print(len(total_tokens))

In [ ]:
text = nltk.Text(total_tokens, name='ARMY')
print(len(set(text.tokens)))
pprint(text.vocab().most_common(10))

In [ ]:
# 여러 리스트들을 하나로 묶어 주는 함수입니다.
def list_to_str(List): 
    return " ".join(List)

## Modeling

In [ ]:
from sklearn.model_selection import train_test_split

y_target = [label for _, label in train_data]
X_features = [list_to_str(doc) for doc, _ in train_data]
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, 
                                                    test_size=0.2,
                                                   random_state=156)

In [ ]:
# Linear classification에 필요한 라이브러리 불러오기
from sklearn.pipeline import Pipeline # 편하게 learner들 만들 수 있다
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
import numpy as np

# 학습 모델 파이프라인 만들기

# 1. Perceptron linear classifier (W^Tx + b)
learner = Pipeline([
    ('vect', CountVectorizer(min_df=5)),
    ('clf', SGDClassifier(loss='perceptron', penalty='l2',
                         alpha=1e-4, random_state=42, 
                         max_iter=500)) # 성능을 높이고싶다면 loss='perceptron'이라면 penalty랑 alpha값 조정
                        #max_iter은 올려도돼(수렴이 안됐다면 더 돌릴 수 있겠지)
])

# 총 4개의 SVC
# 2. SVM with linear kernel
learner2 = Pipeline([
    ('vect', CountVectorizer(min_df=5)),
    ('clf', SVC(kernel='linear')) 
    # 데이터에 noise가 껴있거나 데이터 분포가 선형으로 분류가 불가능한 것이더라도 kernel을 쓰면 선형이 되는 공간으로 옴(머신러닝에서 kernel이라는 공간변환기법쓰는 이유)
    # 단점 : 어떤 kernel을 써야 성능이 가장 좋은지 알 수 없음(해봐야 암), 데이터 분포를 잘 안다면 사용할 수 있는데 대부분 고차원의 데이터 분포를 자세하게 알 수 있을리가 없지
]) 

# 3. SVM with polynomial kernel
learner3 = Pipeline([
    ('vect', CountVectorizer(min_df=5)),
    ('clf', SVC(kernel='poly', degree=8))
]) # 8차원 준 이유는 이 degree도 정해진 게 없어서 높이면 높일 수록 오래 걸리는 대신에 좀 더 많이 구분함

# 4. SVM with Radius Basis Function kernel
learner4 = Pipeline([
    ('vect', CountVectorizer(min_df=5)),
    ('clf', SVC(kernel='rbf'))
])

# 5. SVM with sigmoid kernel
learner5 = Pipeline([
    ('vect', CountVectorizer(min_df=5)),
    ('clf', SVC(kernel='sigmoid'))
])

# 6. Naive Bayes Classifier (Multinomial Ver.)
learner6 = Pipeline([
    ('vect', CountVectorizer(min_df=5)),
    ('clf', MultinomialNB())
])

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score

# X_train, X_test, y_train, y_test 
# 학습기 정하기
classifier = learner

classifier.fit(X_train, y_train)
train_pred = classifier.predict(X_test)
train_accuracy = accuracy_score(y_test, train_pred)
train_f1score = f1_score(y_test, train_pred , average="macro")
train_precision = precision_score(y_test, train_pred, average="macro")
train_recall = recall_score(y_test, train_pred, average="macro")

print("Training Accuracy : %.3f" % train_accuracy)
print("Training F1_score : %.3f" % train_f1score)
print("Training precision : %.3f" % train_precision)
print("Training recall : %.3f" % train_recall)

In [ ]:
classifier = learner2

classifier.fit(X_train, y_train)
train_pred = classifier.predict(X_test)
train_accuracy = accuracy_score(y_test, train_pred)
train_f1score = f1_score(y_test, train_pred , average="macro")

print("Training Accuracy : %.3f" % train_accuracy)
print("Training F1_score : %.3f" % train_f1score)
print("Training precision : %.3f" % train_precision)
print("Training recall : %.3f" % train_recall)

In [ ]:
classifier = learner6

classifier.fit(X_train, y_train)
train_pred = classifier.predict(X_test)
train_accuracy = accuracy_score(y_test, train_pred)
train_f1score = f1_score(y_test, train_pred , average="macro")

print("Training Accuracy : %.3f" % train_accuracy)
print("Training F1_score : %.3f" % train_f1score)
print("Training precision : %.3f" % train_precision)
print("Training recall : %.3f" % train_recall)

### 방법 2

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# LogisticRegreesion의 C는 10으로 설정
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(min_df=5)),
    ('lr_clf', LogisticRegression(C=10))])

# Pipeline 객체를 이용해 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc때문에 수행
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
pred_probs = pipeline.predict_proba(X_test)[:,1]

train_accuracy = accuracy_score(y_test, pred)
train_f1score = f1_score(y_test, pred , average="macro")

print("Training Accuracy : %.3f" % train_accuracy)
print("Training F1_score : %.3f" % train_f1score)

In [ ]:
pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))])

pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
pred_probs = pipeline.predict_proba(X_test)[:,1]

train_accuracy = accuracy_score(y_test, pred)
train_f1score = f1_score(y_test, pred , average="macro")

print("Training Accuracy : %.3f" % train_accuracy)
print("Training F1_score : %.3f" % train_f1score)